In [1]:
import sys
import geopandas as gpd
from shutil import copyfile, move

sys.path.insert(0, './HLB')
import hlb_utils

### Dataset

In [2]:
pre_processing = hlb_utils.PreProcessing('../IMGS/RGB-DSM_test-area_4b.tif', proj_name='area_estudo_teste1')

### Split image [and vectors]

In [3]:
pre_processing.split_img(patch_size=256, overlap=60, save_img=True)

Cortando imagens: 100%|████████████████████████████████████████████████████████████| 3348/3348 [01:58<00:00, 28.23it/s]


Finished!
Total images:  939


In [ ]:
pre_processing.split_vector()

### Make annotation

In [ ]:
pre_processing.grid = gpd.read_file('../datasets/RGB-DTM/img_grid.geojson').set_index('id')

In [ ]:
pre_processing.make_annotations()

### Outros

In [ ]:
import os
from osgeo import gdal, osr
from PIL import Image

shape_path_aux = '../IMGS/TALHAO_ANGULADAS/INF'

result_paths = []
for filename in os.listdir(shape_path_aux):
    result_paths.append(os.path.join(shape_path_aux, filename))

for path in result_paths:
    
    img = Image.open(path)
    w, h = img.size
    img = img.crop((1000, 700, w, h))
    out_path = os.path.join(os.path.dirname(os.path.dirname(path)), 'EDITS', os.path.basename(path))
    img.save(out_path)

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from time import sleep


shape_path_aux = '../datasets/ortofoto_talhao/results'

with tqdm(total=len(os.listdir(shape_path_aux)) as pbar:
    pbar.set_description("Lendo arquivos de entradas")
    sleep(0.1)

    result_paths = []
    for filename in os.listdir(shape_path_aux):
        if os.path.splitext(filename)[1].lower() == '.geojson':
            result_paths.append(os.path.join(shape_path_aux, filename))
        pbar.update(1)
    

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols['geometry'] = pols.buffer(-0.1).buffer(0.1).simplify(0.03)
pols = pols.explode().reset_index(drop=True)
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= 1]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index
# pols['centroid'] = pols['geometry'].centroid


over = gpd.overlay(pols, pols, how='intersection')
over = over.loc[-over['geometry'].is_empty]
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)
delete_lines = []
with tqdm(total=len(over['id_1'])) as pbar:
    pbar.set_description("Simplificando - Micro")
    sleep(0.1)

    for i in over['id_1']:

        intersec = over[over['id_1']==i]

        if intersec.loc[intersec['geometry'].area >= 0.6 * pols.loc[i, 'geometry'].area].any(axis=None) or \
        (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
            delete_lines.append(i)
            
        pbar.update(1)

    delete_lines = (list(set(delete_lines)))

pols.drop(labels=delete_lines, axis=0, inplace=True)
pols.explode().reset_index(drop=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols.to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result.geojson'), driver='GeoJSON')

In [ ]:
pols['id'] = pols.index
pols

In [ ]:
pols.to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result_1.geojson'), driver='GeoJSON')

In [ ]:
over = gpd.overlay(pols, pols, how='intersection')
over = over.loc[-over['geometry'].is_empty]
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)

In [ ]:
over

In [ ]:
delete_lines = []

for i in over['id_1']:
    
    intersec = over[over['id_1']==i]
    
    if intersec.loc[intersec['geometry'].area >= 0.6 * pols.loc[i, 'geometry'].area].any(axis=None) or \
    (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
        delete_lines.append(i)
        
delete_lines = (list(set(delete_lines)))
delete_lines.sort()            

In [ ]:
pols.loc[delete_lines].to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_delete.geojson'), driver='GeoJSON')

In [ ]:
pols.drop(labels=delete_lines, axis=0, inplace=True)
pols

In [ ]:
os.path.dirname(shape_path_aux)

In [ ]:
pols.buffer(-0.1).buffer(0.1).simplify(0.03).to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result_faster_simplify.geojson'), driver='GeoJSON')